**Don't forget to run hydra-setup.ipynb**

In [ ]:
!sbatch --partition=cpu --job-name=prepare-env --ntasks=2 run-hydra-env.sh

In [ ]:
!sbatch --partition=interactive --job-name=prepare-env --ntasks=1 run-hydra-env.sh

In [ ]:
!squeue --user $USER

In [ ]:
%%bash
sbatch --dependency=afterok:513300 --partition=interactive --job-name=optimize --ntasks=8 --ntasks-per-node=8 \
--cpus-per-task=1 --time=00:30:00 --mem-per-cpu=512 --no-requeue --wrap 'srun ./hydra-task.sh --n_workers=8'

In [ ]:
!squeue -p interactive #-u $USER

In [ ]:
!scancel 513324

In [ ]:
!tail -fn100 slurm-513326.out

In [ ]:
import pickle
from matplotlib import pyplot as plt

import hpbandster.visualization as hpvis

with open('workdir/513027.pkl', 'rb') as fh:
    result = pickle.load(fh)

result

In [ ]:
all_runs = result.get_all_runs()
print(len(all_runs))
all_runs[-1]

In [ ]:
# get the 'dict' that translates config ids to the actual configurations
id2conf = result.get_id2config_mapping()
id2conf[(0,0,0)]

In [ ]:
# Here is how you get he incumbent (best configuration)
inc_id = result.get_incumbent_id()
inc_id

In [ ]:
# let's grab the run on the highest budget
inc_runs = result.get_runs_by_id(inc_id)
inc_run = inc_runs[-1]
inc_runs

In [ ]:
# We have access to all information: the config, the loss observed during
#optimization, and all the additional information
inc_loss = inc_run.loss
inc_config = id2conf[inc_id]['config']
inc_test_loss = inc_run.info['validation_loss']

print('Best found configuration:')
print(inc_config)
print('It achieved accuracies of %f (train) and %f (validation).'%(1-inc_loss, 1-inc_test_loss))

In [ ]:
# Let's plot the observed losses grouped by budget,
hpvis.losses_over_time(all_runs)

In [ ]:
# the number of concurent runs,
hpvis.concurrent_runs_over_time(all_runs)

In [ ]:
# and the number of finished runs.
hpvis.finished_runs_over_time(all_runs)

In [ ]:
# This one visualizes the spearman rank correlation coefficients of the losses
# between different budgets.
hpvis.correlation_across_budgets(result)

In [ ]:
# For model based optimizers, one might wonder how much the model actually helped.
# The next plot compares the performance of configs picked by the model vs. random ones
hpvis.performance_histogram_model_vs_random(all_runs, id2conf)